In [81]:
import pandas as pd

In [82]:
df = pd.read_csv("restaurant_df.csv", encoding="UTF-8")

In [83]:
df

,소재지면적,지번주소,도로명주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y)
0,63.02,서울특별시 마포구 서교동 399-9,"서울특별시 마포구 양화로6길 57-12, 1층 (서교동)",라운지목화 합정관,중국식,192669.383195,449648.516083
1,47.98,서울특별시 마포구 서교동 361-6,"서울특별시 마포구 와우산로18길 21, 지1층 (서교동)",소울버튼 스테이지,라이브카페,193201.659573,449878.932066
2,48.37,서울특별시 마포구 서교동 333-37,"서울특별시 마포구 와우산로29마길 7-8, 지1층 우측호 (서교동)",연남골목냉면,한식,193472.105655,450345.430511
3,56.08,서울특별시 마포구 서교동 360-24,"서울특별시 마포구 와우산로 66, 지층 (서교동)",별빛퐁당,한식,193149.006350,449796.117138
4,54.0,서울특별시 마포구 서교동 396-27,"서울특별시 마포구 독막로3길 24-10, 지1층 A호 (서교동)",유아하,중국식,192626.322505,449645.011554
...,...,...,...,...,...,...,...
1487,90.47,서울특별시 마포구 합정동 377-32 동원빌딩,"서울특별시 마포구 양화진4길 17, 동원빌딩 4층 (합정동)",같이양조장,한식,192087.936835,449456.802696
1488,66.00,서울특별시 마포구 합정동 434-1,"서울특별시 마포구 포은로 48, 2층 202호 (합정동)",아더룸,일식,191719.360637,449951.842819
1489,81.80,서울특별시 마포구 합정동 198-27,"서울특별시 마포구 토정로 28-8, 1층 1호,2호 (합정동)",라라와 케이,경양식,192339.304316,449213.586391
1490,68.00,서울특별시 마포구 합정동 200-14,"서울특별시 마포구 토정로 28-7, 1층 (합정동)",데빌스 바지니코,경양식,192356.326751,449219.604108


In [84]:
import time
import random
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [85]:
# Logging 설정
logging.basicConfig(
    filename="crawling.log",  # 로그 파일 저장
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# 랜덤 대기 함수
def random_sleep(min_wait=1, max_wait=3):
    time.sleep(random.uniform(min_wait, max_wait))

# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 백그라운드 실행 파일인데 일단 막아놈
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--log-level=3")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

In [86]:
df["전화번호"] = ""
df["운영시간"] = ""
df["소개"] = ""
df["편의"] = ""
df["주차정보"] = ""


In [87]:
def random_sleep(min_sec, max_sec):
    time.sleep(random.uniform(min_sec, max_sec))

def collect_reviews(driver, df):
    """
    데이터프레임(df)에 '도로명주소', '사업장명' 컬럼이 있다고 가정.
    각 행에 대해 네이버 지도에서 검색 → 리뷰, 편의시설 등 정보를 수집하여
    df의 새 컬럼에 저장 후 최종 CSV 출력.
    """
    
    for index, row in df.iterrows():
        road_address = row["도로명주소"]
        business_name = row["사업장명"]

        # 초기화
        operation_data = {}
        phone = "정보 없음"
        total_reviews = 0
        intro = "정보 없음"
        services = "정보 없음"
        parking = "정보 없음"
        good_points = []
        collected_reviews = []

        try:
            logging.info(f"검색 시작: {business_name} ({road_address})")
            driver.get("https://map.naver.com/")
            random_sleep(2, 4)

            # 검색창 입력
            search_box = driver.find_element(By.XPATH, "//input[contains(@class, 'input_search')]")
            search_box.clear()
            search_box.send_keys(road_address)
            search_box.send_keys(Keys.RETURN)
            random_sleep(3, 5)

            # "더보기" 버튼 클릭
            try:
                more_button = driver.find_element(By.XPATH, "//button[contains(@class, 'link_more')]")
                more_button.click()
                random_sleep(2, 4)
            except NoSuchElementException:
                logging.warning(f"'{business_name}' - '더보기' 버튼 없음, 스킵")
                continue

            # 검색 결과 목록 중에서 사업장명을 포함한 요소 클릭
            place_names = driver.find_elements(By.XPATH, "//strong[contains(@class, 'search_title')]")
            target_place = None

            for place in place_names:
                # strip()으로 앞뒤 공백 제거 후 비교
                if business_name.strip() in place.text.strip():
                    target_place = place
                    break

            if target_place:
                try:
                    search_link = target_place.find_element(By.XPATH, "./ancestor::button[@class='link_search']")
                    search_link.click()
                    time.sleep(3)
                    logging.info(f"'{business_name}' 버튼 클릭 완료!")
                except NoSuchElementException:
                    logging.warning(f"'{business_name}' - 조상 <button> 태그를 찾을 수 없습니다.")
                except Exception as e:
                    logging.warning(f"'{business_name}' 버튼 클릭 실패: {e}")
            else:
                logging.warning(f"'{business_name}' - 검색 결과 없음, 스킵")
                continue

            # iframe 로드 기다린 뒤 진입
            WebDriverWait(driver, 10).until(
                EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe"))
            )
            logging.info("entryIframe 진입 완료")

            # '리뷰' 탭 클릭
            try:
                review_tab = driver.find_element(
                    By.XPATH,
                    "//span[normalize-space(text())='리뷰']"
                )
                review_tab.click()
                logging.info("리뷰 탭 클릭 완료!")
                random_sleep(2, 3)
            except NoSuchElementException:
                logging.warning("'리뷰' 탭을 찾지 못했습니다.")
                continue

            #  "이런점이 좋았어요" 4개 수집
            try:
                items = driver.find_elements(By.XPATH, "//li[contains(@class, 'MHaAm')]")
                for item in items[:4]:
                    label_elem = item.find_element(By.XPATH, ".//span[contains(@class,'t3JSf')]")
                    label_text = label_elem.text.strip()

                    count_elem = item.find_element(By.XPATH, ".//span[contains(@class,'CUoLy')]")
                    count_text = count_elem.text.strip()

                    match = re.search(r'\d+', count_text)
                    count_val = int(match.group()) if match else 0

                    good_points.append([label_text, count_val])
                logging.info(f"'이런점이 좋았어요' 수집 완료: {good_points}")
            except NoSuchElementException:
                logging.warning("'이런점이 좋았어요' 항목을 찾지 못했습니다.")
            except Exception as e:
                logging.warning(f"'이런점이 좋았어요' 수집 중 오류: {e}")

            # 총 리뷰 개수
            try:
                count_elem = driver.find_element(By.XPATH, "//em[@class='place_section_count']")
                count_text = count_elem.text.strip()
                total_reviews = int(count_text)
                logging.info(f"총 리뷰 수: {total_reviews}")
            except NoSuchElementException:
                logging.warning("총 리뷰 수를 찾을 수 없습니다.")
            except ValueError:
                logging.warning(f"리뷰 수 텍스트를 숫자로 변환할 수 없음: {count_text}")

            # 최신순 클릭
            try:
                latest_sort = driver.find_element(By.XPATH, "//a[contains(., '최신순')]")
                latest_sort.click()
                logging.info("최신순 클릭 완료")
                random_sleep(2, 4)
            except Exception:
                logging.warning("최신순 클릭 불가")

            # 리뷰 최대 300개 수집
            MAX_REVIEWS = 300
            while len(collected_reviews) < MAX_REVIEWS:
                review_elements = driver.find_elements(
                    By.XPATH,
                    "//li[contains(@class,'place_apply_pui') and contains(@class,'EjjAW')]"
                )

                new_reviews = []
                for rev in review_elements:
                    try:
                        date_elem = rev.find_element(By.XPATH, ".//time[@aria-hidden='true']")
                        review_date = date_elem.text.strip()
                    except NoSuchElementException:
                        review_date = ""

                    try:
                        text_anchor = rev.find_element(
                            By.XPATH,
                            ".//div[contains(@class,'pui__vn15t2')]//a[@data-pui-click-code='rvshowmore']"
                        )
                        review_text = text_anchor.text.strip()
                    except NoSuchElementException:
                        review_text = ""

                    if review_text:
                        new_reviews.append({
                            "date": review_date,
                            "text": review_text
                        })

                collected_reviews.extend(new_reviews)
                logging.info(f"[진행상황] 현재까지 수집된 리뷰: {len(collected_reviews)}개")

                # 더보기 버튼 클릭
                try:
                    more_button = driver.find_element(
                        By.XPATH,
                        "//a[contains(@class,'fvwqf') and contains(., '더보기')]"
                    )
                    driver.execute_script("arguments[0].scrollIntoView(true);", more_button)
                    more_button.click()
                    logging.info("'더보기' 버튼 클릭 완료!")
                    random_sleep(2, 4)
                except NoSuchElementException:
                    logging.warning("더보기 버튼을 더 이상 찾을 수 없으므로 반복 종료")
                    break

            logging.info(f"최종 수집된 리뷰: 총 {len(collected_reviews)}개")

            # DataFrame에 저장
            df.at[index, "전화번호"] = phone
            df.at[index, "운영시간"] = operation_data
            df.at[index, "총 리뷰 개수"] = total_reviews
            df.at[index, "소개"] = intro
            df.at[index, "편의시설 및 서비스"] = services
            df.at[index, "주차 정보"] = parking
            df.at[index, "이런점이 좋았어요"] = str(good_points)

            # 300개 리뷰를 문자열로
            df.at[index, "최신 300개 리뷰"] = str(collected_reviews[:300])

            logging.info(f"'{business_name}' 데이터프레임 저장 완료")

        except Exception as e:
            logging.error(f"'{business_name}' 크롤링 중 오류 발생: {e}")

    # 브라우저 종료
    driver.quit()

    # CSV로 내보내기
    df.to_csv("updated_naver_map_data.csv", index=False, encoding="utf-8-sig")
    logging.info("크롤링 완료! CSV 파일로 저장됨.")


In [89]:
# -----------------------------------
# 실제 실행 예시
if __name__ == "__main__":
    # 로거 설정
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S"
    )

    # 예시: Selenium 드라이버 준비
    driver = webdriver.Chrome()

    collect_reviews(driver, df)

KeyboardInterrupt: 

In [90]:
df 
# 추가로 알아야할 데이터 형태 
# 이런점이 좋았어요 <- 리스트내 리스트꼴 : [[맛있어요, 100], [친절해요, 50]]
# 최신 300개 리뷰 <- 리스트내 딕셔너리꼴 : [{'date' : 24.10.5 , 'text' : "맛나용 ㅋ"}, {'date' : 24.9.10, 'text' : "별로 ㅋㅋ"}]

,소재지면적,지번주소,도로명주소,사업장명,업태구분명,좌표정보(X),좌표정보(Y),전화번호,운영시간,소개,편의,주차정보,이점좋,총리뷰,최신리뷰,총 리뷰 개수,편의시설 및 서비스,주차 정보,이런점이 좋았어요,최신 300개 리뷰
0,63.02,서울특별시 마포구 서교동 399-9,"서울특별시 마포구 양화로6길 57-12, 1층 (서교동)",라운지목화 합정관,중국식,192669.383195,449648.516083,정보 없음,{},정보 없음,,,,,,291.0,정보 없음,정보 없음,"[['""음식이 맛있어요""', 293], ['""인테리어가 멋져요""', 168], ['...","[{'date': '2.8.토', 'text': '음식이 맛있고 직원분들 친절해요 ..."
1,47.98,서울특별시 마포구 서교동 361-6,"서울특별시 마포구 와우산로18길 21, 지1층 (서교동)",소울버튼 스테이지,라이브카페,193201.659573,449878.932066,,,,,,,,,NaN,NaN,NaN,NaN,NaN
2,48.37,서울특별시 마포구 서교동 333-37,"서울특별시 마포구 와우산로29마길 7-8, 지1층 우측호 (서교동)",연남골목냉면,한식,193472.105655,450345.430511,정보 없음,{},정보 없음,,,,,,38.0,정보 없음,정보 없음,"[['""음식이 맛있어요""', 58], ['""재료가 신선해요""', 15], ['""양이...","[{'date': '24.10.5.토', 'text': '출출해서 급하게 찾아서 왔..."
3,56.08,서울특별시 마포구 서교동 360-24,"서울특별시 마포구 와우산로 66, 지층 (서교동)",별빛퐁당,한식,193149.006350,449796.117138,정보 없음,{},정보 없음,,,,,,12.0,정보 없음,정보 없음,"[['""술이 다양해요""', 9], ['""혼술하기 좋아요""', 8], ['""가성비가 ...","[{'date': '2.1.토', 'text': '종종 들리던 곳이었는데 또 타이밍..."
4,54.0,서울특별시 마포구 서교동 396-27,"서울특별시 마포구 독막로3길 24-10, 지1층 A호 (서교동)",유아하,중국식,192626.322505,449645.011554,정보 없음,{},정보 없음,,,,,,59.0,정보 없음,정보 없음,"[['""음식이 맛있어요""', 56], ['""인테리어가 멋져요""', 36], ['""특...","[{'date': '2.9.일', 'text': '지나가다가 간판이 특이해서 들렀는..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487,90.47,서울특별시 마포구 합정동 377-32 동원빌딩,"서울특별시 마포구 양화진4길 17, 동원빌딩 4층 (합정동)",같이양조장,한식,192087.936835,449456.802696,,,,,,,,,NaN,NaN,NaN,NaN,NaN
1488,66.00,서울특별시 마포구 합정동 434-1,"서울특별시 마포구 포은로 48, 2층 202호 (합정동)",아더룸,일식,191719.360637,449951.842819,,,,,,,,,NaN,NaN,NaN,NaN,NaN
1489,81.80,서울특별시 마포구 합정동 198-27,"서울특별시 마포구 토정로 28-8, 1층 1호,2호 (합정동)",라라와 케이,경양식,192339.304316,449213.586391,,,,,,,,,NaN,NaN,NaN,NaN,NaN
1490,68.00,서울특별시 마포구 합정동 200-14,"서울특별시 마포구 토정로 28-7, 1층 (합정동)",데빌스 바지니코,경양식,192356.326751,449219.604108,,,,,,,,,NaN,NaN,NaN,NaN,NaN


In [72]:
df.columns

Index(['소재지면적', '지번주소', '도로명주소', '사업장명', '업태구분명', '좌표정보(X)', '좌표정보(Y)', '전화번호',
       '운영시간', '소개', '편의', '주차정보', '이점좋', '총리뷰', '최신리뷰', '운영시간_월', '운영시간_화',
       '운영시간_수', '운영시간_목', '운영시간_금', '운영시간_토', '운영시간_일', '총 리뷰 개수', '최신 리뷰 5개',
       '편의시설 및 서비스', '주차 정보', '최신 300개 리뷰', '이런점이 좋았어요'],
      dtype='object')